In [34]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import re
import string
import operator
from nltk.util import ngrams
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier as XG
nltk.download('punkt')
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report as report
from sklearn.ensemble import RandomForestClassifier as RFC

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# read file from google-drive, colab doesnt keeps uploaded files persistently

tweet_file = open(r"train.txt", encoding="utf8")
tweets = tweet_file.read()

# test file separate
tweets_file_test = open(r"test.txt", encoding="utf8")
tweets_test = tweets_file_test.read()

# tweets_list = nltk.sent_tokenize(tweets)
tweets_list = re.split('\n\n',tweets)
tweets_list_test = re.split('\n\n',tweets_test)

tweets_list = tweets_list[:-1]
tweets_list_test = tweets_list_test[:-1]

In [3]:
#clean train dataset
tweets_clean_list = []
sentiments_list = []

for sent in tweets_list:
  # remove \n
  temp = re.sub("\n", ' ', sent)

  # remove ’
  temp = re.sub("\’", '', sent)

  # convert to lowercase
  temp = temp.lower()

  # remove unnecessary punctuations
  temp = " ".join("".join([" " if ch in string.punctuation else ch for ch in temp]).split())
  
  # remove first two words from the message, its metadata
  temp = temp.split(' ', 3)

  # take the next word as sentiment
  sentiment = temp[2]

  # Retreive only the message
  new_msg = temp[3]

  new_msg = new_msg.replace("hin", "")
  new_msg = new_msg.replace("eng", "")
  new_msg = new_msg.replace("o", "")

  # emoji removal by emoji pattern
  emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
  new_msg = emoji_pattern.sub(r'', new_msg)

  # remove multiple consecutive spaces from message
  new_msg = re.sub(' +', ' ', new_msg)

  #after cleaning up, append to new list
  tweets_clean_list.append(new_msg)
  sentiments_list.append(sentiment)

tweets_clean_list[:10]
sentiments_list[:10]

['negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative']

In [4]:
#Clean Test dataset
tweets_clean_list_test = []
sentiments_list_test = []

for sent in tweets_list_test:
  # remove \n
  temp = re.sub("\n", ' ', sent)

  # remove ’
  temp = re.sub("\’", '', sent)

  # convert to lowercase
  temp = temp.lower()

  # remove unnecessary punctuations
  temp = " ".join("".join([" " if ch in string.punctuation else ch for ch in temp]).split())
  
  # remove first three words from the message, its metadata
  temp = temp.split(' ', 3)

  # take the next word as sentiment
  sentiment = temp[2]

  # Retreive only the message
  new_msg = temp[3]

  new_msg = new_msg.replace("hin", "")
  new_msg = new_msg.replace("eng", "")
  new_msg = new_msg.replace("o", "")

  # emoji removal by emoji pattern
  emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
  new_msg = emoji_pattern.sub(r'', new_msg)

  new_msg = re.sub(' +', ' ', new_msg)

  #after cleaning up, append to new list
  tweets_clean_list_test.append(new_msg)
  sentiments_list_test.append(sentiment)


tweets_clean_list_test[:10]


sentiments_list_test[:10]


['neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative']

In [5]:
train_dict = {"Message": tweets_clean_list, "Sentiment": sentiments_list}
df = pd.DataFrame(train_dict)

### removing extra spaces from the tweets

In [6]:
mess = []
for message in df['Message'].values:
    x = re.sub(' +', ' ',message)
    z = x.lstrip()
    y = z.rstrip()
    mess.append(y)

In [7]:
df['Message_clean'] = mess

In [8]:
df = df.drop(['Message'],axis=1)

In [9]:
df.head()

,Sentiment,Message_clean
0,negative,adilnisarbutt pakistan ka ghra tauq he pakista...
1,negative,madarchd mulle ye mathura me nahi dikha tha ja...
2,positive,narendramdi manya pradhan mantri mahday shrima...
3,positive,atheist krishna jcb full trend me chal rahi aa
4,positive,abhisharsharma ravishkumarblg lksabha me janta...


### for test dataset

In [10]:
train_dict = {"Message": tweets_clean_list_test, "Sentiment": sentiments_list_test}
df_test = pd.DataFrame(train_dict)

In [11]:
mess = []
for message in df_test['Message'].values:
    x = re.sub(' +', ' ',message)
    z = x.lstrip()
    y = z.rstrip()
    mess.append(y)

In [12]:
df_test['Message_clean'] = mess

In [13]:
df_test = df_test.drop(['Message'],axis=1)

In [14]:
df_test.head()

,Sentiment,Message_clean
0,neutral,rt uaapcnfessins lve lks gd n maddie ak lang b...
1,neutral,ye ye ye we gnna start anther june n a sur nte...
2,neutral,zwfffy9jgklela1 min f lycg thakurdadu089 manak...
3,negative,caring bht jyada caring curier wale bsdk ke si...
4,positive,alihzaidipti sarfaraza 54 what nnesense kabhi ...


In [15]:
stopwords = set([])
len(tweets_clean_list_test),len(sentiments_list_test)

(1869, 1869)

In [16]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [17]:
# Combining all the above stundents 
from tqdm import tqdm
from bs4 import BeautifulSoup
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df['Message_clean'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.strip())

100%|██████████████████████████████████████████████████████████████████████████| 15131/15131 [00:03<00:00, 4666.71it/s]


In [18]:
df['Message'] = preprocessed_text

In [21]:
pre = preprocessed_text
preprocessed_text = []
for val in pre:
    x = ""
    for w in val.split(' '):
        if len(w)>1:
            x = x+" "+w
    preprocessed_text.append(x)

In [22]:
df['single'] = preprocessed_text

In [23]:
# Combining all the above stundents 
from tqdm import tqdm
from bs4 import BeautifulSoup
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df_test['Message_clean'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.strip())

100%|████████████████████████████████████████████████████████████████████████████| 1869/1869 [00:00<00:00, 4091.43it/s]


In [24]:
df_test['Message'] = preprocessed_text

## Removing all the words with single length

In [26]:
pre = preprocessed_text
preprocessed_text = []
for val in pre:
  #print(val)
    x = ""
    for w in val.split(' '):
        if len(w)>1:
            x = x+" "+w
  #print(x)
  #break
    preprocessed_text.append(x)

In [27]:
df_test['single'] = preprocessed_text

In [28]:
## test data
df_test.head()

,Sentiment,Message_clean,Message,single
0,neutral,rt uaapcnfessins lve lks gd n maddie ak lang b...,rt uaapcnfessins lve lks gd n maddie ak lang b...,rt uaapcnfessins lve lks gd maddie ak lang ba...
1,neutral,ye ye ye we gnna start anther june n a sur nte...,ye ye ye we gnna start anther june n a sur nte...,ye ye ye we gnna start anther june sur nte uh...
2,neutral,zwfffy9jgklela1 min f lycg thakurdadu089 manak...,min f lycg manakgupta fficefknath mein kahna n...,min lycg manakgupta fficefknath mein kahna na...
3,negative,caring bht jyada caring curier wale bsdk ke si...,caring bht jyada caring curier wale bsdk ke si...,caring bht jyada caring curier wale bsdk ke s...
4,positive,alihzaidipti sarfaraza 54 what nnesense kabhi ...,alihzaidipti sarfaraza what nnesense kabhi bay...,alihzaidipti sarfaraza what nnesense kabhi ba...


In [29]:
## train data
df.head()

,Sentiment,Message_clean,Message,single
0,negative,adilnisarbutt pakistan ka ghra tauq he pakista...,adilnisarbutt pakistan ka ghra tauq he pakista...,adilnisarbutt pakistan ka ghra tauq he pakist...
1,negative,madarchd mulle ye mathura me nahi dikha tha ja...,madarchd mulle ye mathura me nahi dikha tha ja...,madarchd mulle ye mathura me nahi dikha tha j...
2,positive,narendramdi manya pradhan mantri mahday shrima...,narendramdi manya pradhan mantri mahday shrima...,narendramdi manya pradhan mantri mahday shrim...
3,positive,atheist krishna jcb full trend me chal rahi aa,atheist krishna jcb full trend me chal rahi aa,atheist krishna jcb full trend me chal rahi aa
4,positive,abhisharsharma ravishkumarblg lksabha me janta...,abhisharsharma ravishkumarblg lksabha me janta...,abhisharsharma ravishkumarblg lksabha me jant...


In [ ]:
## single is a column containing only the words having length greater then one

### Applying XGBoost

In [32]:
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(df['single'])
X_test = tfidf.transform(df_test['single'])

In [ ]:
y_test1 = df_test['Sentiment']
y_test = []
for val in y_test1:
    if val=='neutral':
        y_test.append(1)
    if val=='positive':
        y_test.append(2)
    if val=='negative':
        y_test.append(3)

y_test1 = df['Sentiment']
y_train = []
for val in y_test1:
    if val=='neutral':
        y_train.append(1)
    if val=='positive':
        y_train.append(2)
    if val=='negative':
        y_train.append(3)

In [51]:
model = XG(max_depth=6)
model.fit(X_train,y_train)
y_pre = model.predict(X_test)
print("Accuracy Score of the model:",accuracy_score(y_test,y_pre))

Accuracy Score of the model: 0.555912252541466


In [59]:
print("Accuracy score of the model :",accuracy_score(y_test,y_pre))

Accuracy score of the model : 0.555912252541466


#### Representation of classes:
- 1 : Neutral
- 2 : Positive
- 3 : Negative

In [53]:
result = report(y_test,y_pre).split("\n")
for d in result:
    print(d)

              precision    recall  f1-score   support

           1       0.51      0.64      0.57       754
           2       0.59      0.54      0.56       582
           3       0.63      0.45      0.53       533

    accuracy                           0.56      1869
   macro avg       0.57      0.54      0.55      1869
weighted avg       0.57      0.56      0.55      1869



### Applying Random Forest

In [77]:
model1 = RFC(n_estimators=120,max_depth=28)
model1.fit(X_train,y_train)
y_pre = model1.predict(X_test)
print("Accuracy score of the model :",accuracy_score(y_test,y_pre))

Accuracy score of the model : 0.5436062065275549


In [78]:
result = report(y_test,y_pre).split("\n")
for d in result:
    print(d)

              precision    recall  f1-score   support

           1       0.49      0.67      0.57       754
           2       0.58      0.55      0.57       582
           3       0.66      0.35      0.46       533

    accuracy                           0.54      1869
   macro avg       0.58      0.53      0.53      1869
weighted avg       0.57      0.54      0.54      1869



In [79]:
print("Accuracy score of the model :",accuracy_score(y_test,y_pre))

Accuracy score of the model : 0.5436062065275549


In [ ]:
### Optimal set of features using PCA:

In [87]:
from sklearn.decomposition import TruncatedSVD as TSVD

In [90]:
X_train.shape

(15131, 37503)

### Optimal set of features using TruncatedSVD

In [ ]:
for i in range(500,37503,2000):
    tvd = TSVD(n_components=i)
    tvd.fit(X_train)  
    print("Explained Variance : ",tvd.explained_variance_ratio_.sum())

Explained Variance :  0.3150661153412167
Explained Variance :  0.6143385083197318
